In [10]:
import pandas as pd
import re, os
import numpy as np
import tool
import nlptools as nlpt
import dataprocess as dp

from unidecode import unidecode
import io, os

import warnings

warnings.simplefilter("ignore")  


from dotenv import load_dotenv

load_dotenv()
CONFIG_PATH = os.getenv('CONFIG_PATH')

def adjust_intitule(x, dict_etoile, dict_rdm, status_bar= None):
    if status_bar is not None:
        status_bar.increment()
        status_bar.display()

    intitule = x['INTITULE']
  
    if x['MENTION SPECIFIQUE'] in list(dict_etoile.keys()):
        intitule = intitule + dict_etoile.get(x['MENTION SPECIFIQUE'])

    x['INTITULE'] = intitule
    return x

def get_nat_index(gencod, df):
    # S'assurer que gencod est une chaîne de caractères avant d'utiliser startswith
    indexes = df[df['GENCOD'].apply(lambda x: gencod in str(x))].index
    return indexes

def set_nat(df_nat, col, x, status_bar):
    if status_bar is not None:
        status_bar.increment()
        status_bar.display()

    # Utiliser .loc avec des crochets [] au lieu de parenthèses
    indexes = get_nat_index(x['GENCOD'], df_nat)
    if not indexes.empty:
        return df_nat[col].loc[indexes].iloc[0]
    else:
        return x[col]  
    
def set_lot_virtuel(x, market):
    if x['NB UC DS LOT VIRTUEL'] == 3:
        return " 2 acheté, le 3ème offert**"
    elif x['NB UC DS LOT VIRTUEL'] == 2:
        if market:
            reste = int(x['LOT VIRTUEL VALEUR']*100)%100
            return f"1 acheté = le 2ème à -{int(x['LOT VIRTUEL VALEUR'])}€*.{'0'+str(reste) if reste <10 else reste}".upper() #.replace('.',',')
        else:
            pourc = int(100*x['LOT VIRTUEL VALEUR']/x['PVC MAXI'])
            pourc = min([20, 25, 34, 50, 60, 68, 75, 80], key=lambda x: abs(x - pourc))
            return f"1 acheté = le 2ème à -{pourc}%*".upper()
    else:
        return ''
    
def set_pvcnet_lot_virtuel(x):
    if not tool.isnull(x['NB UC DS LOT VIRTUEL']) and int(x['NB UC DS LOT VIRTUEL']) in[2,3]:
        return float(f"{(x['PRIX LOT AVEC REMISE']/x['NB UC DS LOT VIRTUEL']):.2f}")
    else:
        return x['PVC NET']
    
    
def set_pxkg_lotvirtuel(x, status_bar=None):
    if status_bar is not None:
        status_bar.increment()
        status_bar.display()
    if x['LOT VIRTUEL'] == '':
        return ''
    else:
        label = f"Les {int(x['NB UC DS LOT VIRTUEL'])} : {np.round(x['PRIX LOT AVEC REMISE'],2):.2f} €  \nAu lieu de {np.round(x['PRIX LOT INIT'],2):.2f} €  \nSoit le {'kg' if x['K/L']=='K' else 'litre' } : {np.round(x['PRIX AU KG OU L DU LOT VIRTUEL'],2):.2f} €  \nÉconomies : {np.round(x['LOT VIRTUEL VALEUR'],2):.2f} €"
        label = label.replace('.',',')
        return label

def get_market_tag(df, df_m, tag_l):
    # Boucle sur chaque tag
    for tag in tag_l:
        # Création de la colonne 'MARKET_<tag>' si elle n'existe pas
        market_col = f'MARKET_{tag}'
        if tag == 'PVC MAXI':
            market_col = 'MARKET_PVC_MAXI'
        if market_col not in df.columns:
            df[market_col] = None       
        # Mapper les valeurs de df_m[tag] basées sur la clé 'gencod'
        mapping = df_m.set_index('GENCOD')[tag]
        
        # Appliquer le mapping et remplir les valeurs manquantes avec None
        df[market_col] = df['GENCOD'].map(mapping).where(df['GENCOD'].notna(), '')
        df[market_col] = df[market_col].fillna('')
    
    return df

def set_px_kg(x): 
    if x['K/L'] in ['K','L']:
        unit = 'kg' if x['K/L']=='K' else 'litre'
        if x['K/L']=='L' and int(x['POIDS/VOLUME'])==1:
            return ''
        if tool.isnull(x['RI EN %']) and tool.isnull(x['RI EN €']):
            return f"Soit le {unit} : {x['PRIX AU KG OU L']:.2f} €".replace('.', ',')
        else:
            return f"Soit le {unit} : {x['PRIX NET AU KG OU L']:.2f} € \nAu lieu de {x['PRIX AU KG OU L']:.2f} €".replace('.', ',')
    else:
        return ''

def apply_segu_process(self, df, reset=False, market = False):
  
    load_dotenv()
    CONFIG_FILE = os.getenv('CONFIG_FILE')

    columns = df.columns
    len_df = len(df)

    output_col = dp.output_col('SEGU_M' if market else 'SEGU')

    if not reset:
        df.drop(['theme', 'radio', 'réseaux sociaux/digital'], axis=1, inplace=True)
        rename_dict = {'PVC COCCINELLE': 'PVC MAXI','PVC COCCIMARKET': 'PVC MAXI', 'prix net':'PVC NET','affiche':'AFFICHE','une':'PRODUIT DE UNE',               ###### dans liste client
                        'der': 'PRODUIT EN DER', 'mise en avant':'MISE EN AVANT', 'Picto': 'PICTO', 'Catégorie': 'CATEGORIE',
                        'SUPER':'FORMAT GV', 'EXPRESS':'FORMAT MV', 'CMK': 'FORMAT MVK', 'origine' : 'ORIGINE',
                        'bonus':'BONUS', 'RI valeur':'RI EN €', 'RI %':'RI EN %',
                        'libellé remise': 'LOT VIRTUEL'}
        df = df.rename(rename_dict, axis=1)
        rename_dict = {col : unidecode(col.upper()) for col in df.columns}
        rename_dict['pageligne']='pageligne'
        rename_dict['RI EN €']='RI EN €'
        df = df.rename(rename_dict, axis=1)
        df = df.copy()
        columns = df.columns

        df[[col for col in output_col if col not in columns]] = None # ajouter les colonnes manquantes
       

    df['CODE OP'] = self.code_op
    df['DATE OP'] = tool.set_month_in_french(tool.get_d_label(tool.extract_dates(self.date_op)))
    if not reset: 
        df['SELECTION SEGUREL'] = df['pageligne'].apply(lambda x: 'X' if x in self.df_prod_seg['pageligne'].to_list() else '')
        df['SELECTION FRANCAP'] = df['pageligne'].apply(lambda x: 'X' if x not in self.df_prod_seg['pageligne'].to_list() else '')       
        df['SR'] = 'X'    # Mettre tout les produits en stop rayon par default
        df['CATALOG'] = 'X' 
    ####### transformer les colonnes booléènnes (celle qui utilise X pour Vrai)
    col_bool = dp.col_bool('SEGU_M' if market else 'SEGU')
    print(df[col_bool])
    for col in col_bool:
        df[col] = df[col].apply(lambda x : True if str(x).strip() in ['X', 'x'] else False)    
    df = dp.verifGencod(None, df, len_df) #Verification et formattage des GENCOD
    if not market:
        df = df.apply(lambda x: dp.dissociatephoto(x,None), axis=1)
        for i in range(8):
            df[f'PHOTO{i+1}'] = df[f'PHOTO{i+1}'].apply(lambda x: '' if tool.isnull(x) else str(x).split('.')[0]).fillna('').astype(str)
        df['ISFROM_LOGO'] = df.apply(lambda x :dp.get_logo_path(x),axis=1)
        #df = df.copy()
        #status_bar.display(float(1))
    categories = pd.read_excel(CONFIG_FILE, sheet_name="CATEGORIES", header=0)
    match_categories = pd.read_excel(CONFIG_FILE, sheet_name="MATCH_CATEGORIE", header=0)
    match_mentions = pd.read_excel(CONFIG_FILE, sheet_name="MATCH_MENTION", header=0)
    mentions = pd.read_excel(CONFIG_FILE, sheet_name="MENTIONS", header=0)
    match_cat = {lab.lower():cat for lab, cat in zip(match_categories.INTITULE, match_categories.CATEGORIE)}
    match_men = {lab.lower():men for lab, men in zip(match_mentions.INTITULE, match_mentions.MENTION)}
    categories_avec_mention = categories[pd.notna(categories.MENTION)]
    cat_men = dict(zip(categories_avec_mention.CATEGORIE, categories_avec_mention.MENTION ))
    men_etoile = dict(zip(mentions.MENTION, mentions.NOTE ))
    #status_bar = tool.ProgressBar(status, len_df, '......ajustement des decriptifs des produits francap')
    df['DESCRIPTIF'] = df.apply(lambda x: dp.correct_descriptif(str(x['DESCRIPTIF'])), axis =1)
    #status_bar.display(float(1))        
    
    #status_bar = tool.ProgressBar(status, len_df, '......assignation des categories des produits francap')
    df['CATEGORIE'] = df.apply(lambda x: dp.correct_categorie(x,categories.CATEGORIE.to_list(), match_cat, None), axis =1)
    #status_bar.display(float(1))
    #status_bar = tool.ProgressBar(status, len_df, '......assignation des mention spécifiques des produits francap')
    df['MENTION SPECIFIQUE'] = df.apply(lambda x: dp.correct_mention(x,mentions.MENTION.to_list(),cat_men, match_men), axis=1)
    #status_bar.display(float(1))
    
    #status_bar = tool.ProgressBar(status, len_df, '......assignation des intitulés des produits francap')
    df['INTITULE'] = df.apply(lambda x: dp.clean_intitule(x['INTITULE']), axis =1)
    df = df.apply(lambda x: x if x['SELECTION FRANCAP'] else adjust_intitule(x, men_etoile, {}, None), axis=1)
    #status_bar.display(float(1))
    #status_bar = tool.ProgressBar(status, len_df, '......assignation des pictos des produits francap')
    df['PICTO'] = df['PICTO'].apply(lambda x: '' if tool.isnull(x) else x).fillna('').astype(str)
    df['ORIGINE'] = df.apply(lambda x: '' if x['PICTO']=='SURGELÉS' else x['ORIGINE'], axis=1)
    df['ORIGINE'] = df['ORIGINE'].apply(lambda x: '' if tool.isnull(x) else x).fillna('').astype(str)
    #status_bar.display(float(1))
    df['ORIGINE'] = df.apply(lambda x: dp.correct_origine(x), axis =1)
    #status_bar.display(float(1))
    
    #status_bar = tool.ProgressBar(status, len_df, '......corrections des marques')
    marques = pd.read_excel(CONFIG_FILE, sheet_name="MARQUES", header=0)
    for n_init, n_final in zip(marques.Nom, marques.Correction):
        df.MARQUE = df.MARQUE.fillna('').astype('str').apply(lambda x: re.sub(str(n_init), str(n_final), str(x)).upper())
        #status_bar.increment()
        #status_bar.display()

    #LOT VIRTUEL################################################
    df['LOT VIRTUEL'] = df.apply(lambda x: set_lot_virtuel(x, market), axis=1)
    ####################### Nettoyage PRIX AU KG DU LOT VIRTUEL
    df['PRIX AU KG DU LOT VIRTUEL'] = df.apply(lambda x: set_pxkg_lotvirtuel(x, None), axis=1)
    df['PRIX AU KG'] = df.apply(lambda x: set_px_kg(x),axis=1)
    ####AFFICHE
    if not reset:
        df['PVC MAXI'] = df['PVC MAXI'].apply(lambda x: None if x is None or x in ['',' '] else float(x)) ### PVC MAXI
        df['PVC NET'] = df['PVC NET'].apply(lambda x: None if x is None or x in ['',' '] else float(x)) ### PVC MAXI
        df['PVC NET'] = df.apply(lambda x: set_pvcnet_lot_virtuel(x), axis=1)
        df['BONUS'] = df.apply(lambda x: x.BONUS if pd.isna(x['BONUS %']) or x['BONUS %'] == '' or x['BONUS %'] is None else x['BONUS %'], axis=1)
        df['PRODUIT DE UNE'] = df['PRODUIT DE UNE'].apply(lambda x: '' if tool.isnull(x) else x).fillna('').astype(str)
        df['PRODUIT EN DER'] = df['PRODUIT EN DER'].apply(lambda x: '' if tool.isnull(x) else x).fillna('').astype(str)
        df['MISE EN AVANT'] = df['MISE EN AVANT'].apply(lambda x: '' if tool.isnull(x) else x).fillna('').astype(str)
    if not market:
        tag = ['DESCRIPTIF', 'PRIX AU KG', 'PVC MAXI', 'BONUS', 'RI EN €', 'RI EN %', 'PVC NET', 'LOT VIRTUEL', 'PRIX AU KG DU LOT VIRTUEL','SR','CATALOG']
        df = get_market_tag(df, self.main_df_m, tag)
        df = df.rename({'MARKET_SR': 'SR_SEGU_MARKET'})
        df['FORMAT MVK'] = df['GENCOD'].apply(lambda x: x in self.main_df_m['GENCOD'].values)


    return df[output_col]

In [ ]:

class Operation:
   
    def __init__(self):
        self.main_df = None
        self.code_op, self.date_op = None, None
        self.st = None
      
    def detach_session(self):
        self.st = None

    #def get_new_list_from_customer(self, liste_nat, liste_codi, liste_segu):
#
    #    self.df_nat = pd.read_excel(liste_nat, sheet_name=0, header=0)
    #    natsheets = pd.ExcelFile(liste_nat).sheet_names
    #
#
    #    if   'A' in natsheets:
    #        self.df_nat = pd.read_excel(liste_nat, sheet_name='A', header=0)
    #    elif 'FINALE'in natsheets:
    #        self.df_nat = pd.read_excel(liste_nat, sheet_name='FINALE', header=0)
#
    #            # Garder uniquement les lignes où 'colonne_a_filtrer' n'est ni NaN ni une chaîne vide
    #    self.df_nat = self.df_nat[self.df_nat['GENCOD'].notna() & (self.df_nat['GENCOD'] != '')]
#
    #    #self.df_segu = self.df_segu[self.df_segu['GENCOD'].notna() & (self.df_segu['GENCOD'] != '')]
    #    #self.main_df = dp.apply_base_process(self.st, self.df_nat, 'NAT')
    #    self.code_op, self.date_op = self.df_nat['CODE OP'][0], self.df_nat['DATE OP'][0]


class Segurel(Operation):
    def __init__(self):
        # Appelle le constructeur de la classe parent
        super().__init__()
        self.main_df_m = None
        self.df_segu, self.df_segu_m = None, None
        self.df_non, self.df_prod_seg = None, None

    def get_new_list_from_customer(self, liste_segu):
        segusheets = pd.ExcelFile(liste_segu).sheet_names

        assert(all(sheet in [s.strip() for s in segusheets] for sheet in ['EP','COCCINELLE agence','CMK agence'])),\
              f" La feuilles 'EP','COCCINELLE agence' ou 'CMK agence' est manquante dans le fichier SEGUREL: {segusheets}"
 
        for sheet in segusheets:
            if sheet.strip() == 'COCCINELLE agence':
                cocci = sheet
            elif sheet.strip() == 'CMK agence':
                cmk = sheet
            elif sheet.strip() == 'EP':
                ep = sheet

        
    ##########################COCCINELLE
        i=0
        header_cond = True
        while (header_cond):
            self.df_segu = pd.read_excel(liste_segu, sheet_name=cocci, header=i)
            header_cond = 'pageligne' not in [str(col).strip() for col in list(self.df_segu.columns)]
            i +=1
        
        self.df_segu = self.df_segu.dropna(subset=['pageligne'])
        #self.df_segu = self.df_segu.rename(columns = { col : col.strip() for col in list(self.df_segu.columns)})
        rename_dict = {}
        col2remove = {}
        col2add = {}
        self.df_segu = dp.clean(self.df_segu, rename_dict, col2remove, col2add)

    ##########################MARKET
        i=0
        header_cond = True
        while (header_cond):
            self.df_segu_m = pd.read_excel(liste_segu, sheet_name=cmk, header=i)
            header_cond = 'pageligne' not in [str(col).strip() for col in list(self.df_segu_m.columns)]
            i +=1
        #
        self.df_segu_m = self.df_segu_m.dropna(subset=['pageligne'])
        #self.df_segu_m = self.df_segu_m.rename(columns = { col : col.strip() for col in list(self.df_segu_m.columns)})
        self.df_segu_m = dp.clean(self.df_segu_m, rename_dict, col2remove, col2add)
           

#    ##########################PROD SEGUREL
        i=0
        not_header_cond = True
        while (not_header_cond):
            df = pd.read_excel(liste_segu, sheet_name=ep, header=i).dropna(axis=1, how='all')
            not_header_cond = 'pageligne' not in [str(col).strip() for col in list(df.columns)]
            i +=1
        self.df_prod_seg = df.dropna(subset=['pageligne']).reset_index(drop=True)

    def init_main(self, st=None, reference = None):
        self.st = st
        if reference is None:
            self.main_df = self.df_segu
            self.main_df_m = self.df_segu_m
        else:
            self.main_df = reference.main_df
            self.main_df_m = reference.main_df_m

    def set_main(self, st=None):
        if st is not None:
            self.st = st
        
        self.main_df_m = apply_segu_process(self, self.df_segu_m, market=True).reset_index(drop=True)
        self.main_df = apply_segu_process(self, self.df_segu, market =False).reset_index(drop=True)

        self.main_df.CLE = self.main_df.index + 1

    def reset_main(self, st=None):
        if st is not None:
            self.st = st
        
        self.main_df = self.main_df[self.main_df['GENCOD'].notna() & (self.main_df['GENCOD'] != '')]



    def save(self, filepath):
        try:
            self.detach_session()
            with open(filepath, 'wb') as file:
                pickle.dump(self, file)
                return 'ok'
        except Exception as e:
                return e
        


    def prepare_to_excel(self):
        
        df = self.main_df[dp.output_col('SEGU')]
        #df_m = self.main_df_m[dp.output_col('SEGU')]
              
        col_bool = dp.col_bool('SEGU')
        for col in col_bool:
            df[col] = self.main_df[col].apply(lambda x : 'X' if x else '')
            #df_m[col] = self.main_df_m[col].apply(lambda x : 'X' if x else '')

        return df

    def set_worksheet(self, worksheet, workbook, df):
        # Définir le format pour l'en-tête (gras + background bleu ciel)
        internal_f = workbook.add_format({ 'bg_color': '#E9DACE', 'font_color': '#E9DACE',  'text_wrap': True, 'valign': 'top'})  ##### Variable interne  couleur specifique à Codi
        format1 = workbook.add_format({'text_wrap': True, 'valign': 'top'}) #colonne neutre sans transfo
        useless_f = workbook.add_format({'font_color':'#000000' , 'bg_color': '#000000'}) # colonne inutile à supprimer
        bool_f = workbook.add_format({'align': 'center', 'valign': 'top'}) #gris
        final_bool_f = workbook.add_format({'align': 'center', 'font_color': '#3470C9', 'bold': True, 'valign': 'top'})
        toverify_bool_f = workbook.add_format({'align': 'center', 'font_color': '#3470C9', 'bold': False, 'valign': 'top'})
        toverify_f = workbook.add_format({'font_color': '#3470C9', 'bold': False, 'valign': 'top'})  #bleu
        toverify_wf = workbook.add_format({'font_color': '#3470C9', 'bold': False, 'text_wrap': True, 'valign': 'top'})  #bleu
        final_f = workbook.add_format({'font_color': '#3470C9', 'bold': True, 'valign': 'top'})  #bleu
        final_wf = workbook.add_format({'font_color': '#3470C9', 'bold': True, 'text_wrap': True, 'valign': 'top'})  #bleu
        format_w = workbook.add_format({'text_wrap': True, 'valign': 'top'}) # affiche le retour chariot
        # Définir le format pour l'en-tête (gras + background bleu ciel)
        header_f = workbook.add_format({'text_wrap': True,  
                                        'bold': True,       # Texte en gras
                                        'valign': 'top'     # Aligné en haut de la cellule
                                    })
        # Appliquer le format à la première ligne (les en-têtes)
        worksheet.set_row(0, 40, header_f)
        for col, value in enumerate(dp.output_col('SEGU')):  # On part de 1 pour ignorer les en-têtes
            worksheet.write(0, col, value, header_f)
        worksheet.freeze_panes(1, 0)
        for row_num, value in enumerate(df[dp.output_col('SEGU')[0]].values, 1):  # On part de 1 pour ignorer les en-têtes
            worksheet.write(row_num, 0, value, internal_f)
        worksheet.set_column('B:B', 5, toverify_f)  ##### CLE
        worksheet.set_column('C:C', 10, final_f)  ##### CODE OP
        worksheet.set_column('D:D', 20, final_wf)  ##### DATE OP
        worksheet.set_column('E:E', 10, internal_f)  #####  MENTION SPECIFIQUE
        worksheet.set_column('F:F', 20, toverify_wf)  ##### CATEGORIE
        worksheet.set_column('G:G', 20, toverify_wf)  ##### INTITULE
        worksheet.set_column('H:H', 20, final_f)  ##### GENCOD
        worksheet.set_column('I:I', 20, toverify_f)  ##### MARQUE
        worksheet.set_column('J:J', 20, format1)  ##### ORIGINE
        worksheet.set_column('K:K', 20, internal_f)  ##### MECAPROMO
        worksheet.set_column('L:L', 10, bool_f)  ##### RONDE DES MARQUES
        worksheet.set_column('M:M', 40, format1)  ##### DESCRIPTIF     
        worksheet.set_column('N:N', 20, toverify_f)  ##### PRIX AU KG
        worksheet.set_column('O:O', 20, toverify_f)  ##### PRIX AU KG
        worksheet.set_column('P:P', 10, toverify_f)  ##### PVC MAXI
        worksheet.set_column('Q:Q', 10, format1)  ##### BONUS
        worksheet.set_column('R:R', 10, format1)  ##### RI EN €
        worksheet.set_column('S:S', 10, format1)  ##### RI EN %
        worksheet.set_column('T:T', 10, format1)  ##### PVC NET
        worksheet.set_column('U:U', 20, toverify_wf)  ##### LOT VIRTUEL
        worksheet.set_column('V:V', 20, toverify_wf)  ##### PRIX AU KG DU LOT VIRTUEL
        worksheet.set_column('W:W', 40, format1)  ##### MARKET DESCRIPTIF     
        worksheet.set_column('X:X', 20, toverify_f)  ##### MARKET PRIX AU KG
        worksheet.set_column('Y:Y', 20, toverify_f)  ##### MARKET PRIX AU KG
        worksheet.set_column('Z:Z', 10, toverify_f)  ##### MARKET PVC MAXI
        worksheet.set_column('AA:AA', 10, format1)  ##### MARKET BONUS
        worksheet.set_column('AB:AB', 10, format1)  ##### MARKET RI EN €
        worksheet.set_column('AC:AC', 10, format1)  ##### MARKET RI EN %
        worksheet.set_column('AD:AD', 10, format1)  ##### MARKET PVC NET
        worksheet.set_column('AE:AE', 20, toverify_wf)  ##### MARKET LOT VIRTUEL
        worksheet.set_column('AF:AF', 20, toverify_wf)  ##### MARKET PRIX AU KG DU LOT VIRTUEL
        worksheet.set_column('AG:AG', 10, final_bool_f)  ##### FORMAT GV
        worksheet.set_column('AH:AH', 10, final_bool_f)  ##### FORMAT MV
        worksheet.set_column('AI:AI', 10, final_bool_f)  ##### FORMAT MVK
        worksheet.set_column('AJ:AJ', 10, bool_f)  ##### FORMAT PV
        worksheet.set_column('AK:AK', 10, useless_f)   ##### VIDE_2                 ##### PRODUIT DE UNE
        worksheet.set_column('AL:AL', 10, format1)  ##### PRODUIT EN DER
        worksheet.set_column('AM:AM', 10, format1)  ##### MISE EN AVANT
        worksheet.set_column('AN:AN', 10, bool_f) #### SR SEGU MARKET
        worksheet.set_column('AO:AO', 20, final_f)  ##### PICTO
        worksheet.set_column('AP:AP', 5, final_bool_f)  ##### SR
        worksheet.set_column('AQ:AQ', 10, final_bool_f)  ##### AFFICHE
        worksheet.set_column('AR:AR', 10, useless_f)  ##### INFO COMPLEMENTAIRES 
        worksheet.set_column('AS:AS', 10, toverify_bool_f) ##### SELECTION FRANCAP
        worksheet.set_column('AT:AT', 10, toverify_bool_f)  ##### SELECTION CODIFRANCE
        worksheet.set_column('AU:AU', 20, final_f)  ##### INFO COMPLEMENTAIRES
        worksheet.set_column('AV:AV', 20, final_f)  ##### PHOTO1
        worksheet.set_column('AW:AW', 20, final_f)  ##### PHOTO2
        worksheet.set_column('AX:AX', 20, final_f)  ##### PHOTO3
        worksheet.set_column('AY:AY', 20, final_f)  ##### PHOTO4
        worksheet.set_column('AZ:AZ', 20, final_f)  ##### PHOTO5
        worksheet.set_column('BA:BA', 20, final_f)  ##### PHOTO6
        worksheet.set_column('BB:BB', 20, final_f)  ##### PHOTO7
        worksheet.set_column('BC:BC', 20, final_f)  ##### PHOTO8
        worksheet.set_column('BD:BD', 20, format1)  ##### DESCRIPTIF_2
        worksheet.set_column('DE:DE', 20, format1)  ##### PRIX AU KG_2
        worksheet.set_column('DF:DF', 20, format1)  ##### PRIX AU KG DU LOT VIRTUEL_2
        worksheet.set_column('DG:DG', 20, format1)  ##### DESCRIPTIF_3
        worksheet.set_column('DH:DH', 20, format1)  ##### PRIX AU KG_3
        worksheet.set_column('BI:BI', 20, format1)  ##### PRIX AU KG DU LOT VIRTUEL_3
        worksheet.set_column('BJ:BJ', 20, format1)  ##### DESCRIPTIF_4
        worksheet.set_column('BK:BK', 20, format1)  ##### PRIX AU KG_4
        worksheet.set_column('BL:BL', 20, format1)  ##### PRIX AU KG DU LOT VIRTUEL_4
        worksheet.set_column('BM:BM', 20, format1)  ##### DESCRIPTIF_5
        worksheet.set_column('BN:BN', 20, format1)  ##### PRIX AU KG_5
        worksheet.set_column('BO:BO', 20, format1)  ##### PRIX AU KG DU LOT VIRTUEL_5
        worksheet.set_column('BP:BP', 20, format1)  ##### DESCRIPTIF_6
        worksheet.set_column('BQ:BQ', 20, format1)  ##### PRIX AU KG_6
        worksheet.set_column('BR:BR', 20, format1)  ##### PRIX AU KG DU LOT VIRTUEL_6
        worksheet.set_column('BS:BS', 15, format1)  ##### SUPER_Page
        worksheet.set_column('BT:BT', 15, format1)  ##### SUPER_Rang
        worksheet.set_column('BU:BU', 15, format1)  ##### SUPER_Case
        worksheet.set_column('BV:BV', 15, format1)  ##### EXPRESS_Page
        worksheet.set_column('BW:BW', 15, format1)  ##### EXPRESS_Rang
        worksheet.set_column('BX:BX', 15, format1)  ##### EXPRESS_Case
        worksheet.set_column('BY:BY', 15, format1)  ##### MARKET_Page
        worksheet.set_column('BZ:BZ', 15, format1)  ##### MARKET_Rang
        worksheet.set_column('CA:CA', 15, format1)  ##### MARKET_Case
        worksheet.set_column('CB:CB', 15, format1)  ##### REGIO_Page
        worksheet.set_column('CC:CC', 15, format1)  ##### REGIO_Rang
        worksheet.set_column('CD:CD', 15, format1)  ##### REGIO_Case
        worksheet.set_column('CE:CE', 15, format1)  ##### SUPER_WP
        worksheet.set_column('CF:CF', 15, format1)  ##### EXPRESS_WP
        worksheet.set_column('CG:CG', 15, format1)  ##### MARKET_WP
        return worksheet

    def set_excel_file(self, output):
        with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
            df_output = self.prepare_to_excel()
            df_output.to_excel(writer, sheet_name='COCCINELLE', index=False)
            #df_output_m.to_excel(writer, sheet_name='COCCIMARKET', index=False)            

            # Obtenir l'objet workbook et worksheet
            workbook  = writer.book
            worksheet = writer.sheets['COCCINELLE']
            #worksheet_m = writer.sheets['COCCIMARKET']
            
            worksheet = self.set_worksheet(worksheet, workbook, df_output)
            #worksheet_m = self.set_worksheet(worksheet_m, workbook, df_output_m)

            output.seek(0)
            with open('formatted_output.xlsx', 'wb') as f:
                f.write(output.getvalue())

        return output

In [12]:
segurel = Segurel()
segurel.code_op = 'PR2507'
segurel.date_op = f"Du 18/03/2025 au 04/04/2025"
segu_file = "/Users/stevecostalat/Downloads/FRANCAP/LISTE_SEGU_2507.xlsx"
segurel.get_new_list_from_customer(segu_file)

segurel.init_main()
segurel.set_main()


    MISE EN AVANT SELECTION FRANCAP SELECTION SEGUREL SR MISE EN AVANT  \
0               X                 X                    X             X   
1               0                 X                    X             0   
2               0                 X                    X             0   
3               0                 X                    X             0   
4               0                 X                    X             0   
..            ...               ...               ... ..           ...   
162             0                 X                    X             0   
163             0                 X                    X             0   
164             X                 X                    X             X   
165           NaN                 X                    X           NaN   
166           NaN                 X                    X           NaN   

    CATALOG CATALOG_MARKET  
0         X           None  
1         X           None  
2         X           No

In [34]:
print(segurel.main_df.columns)
df = segurel.main_df
df_g = df.groupby(['CATEGORIE', 'MARQUE', 'PVC MAXI']) \
         .apply(lambda x: list(x.index)) \
         .reset_index(name='LISTE_INDEX')
df_g = df_g[df_g['LISTE_INDEX'].apply(len)>1]
df_g['maxindex']=df_g['LISTE_INDEX'].apply(max)
df_g = df_g.sort_values(by='maxindex', ascending=False)

len(df_g)

Index(['ISFROM_LOGO', 'CLE', 'CODE OP', 'DATE OP', 'MENTION SPECIFIQUE',
       'CATEGORIE', 'INTITULE', 'GENCOD', 'MARQUE', 'ORIGINE', 'MECAPROMO',
       'RONDE DES MARQUES', 'DESCRIPTIF', 'PRIX AU KG', 'PVC MAXI', 'BONUS',
       'RI EN €', 'RI EN %', 'PVC NET', 'LOT VIRTUEL',
       'PRIX AU KG DU LOT VIRTUEL', 'MARKET_DESCRIPTIF', 'MARKET_PRIX AU KG',
       'MARKET_PVC_MAXI', 'MARKET_BONUS', 'MARKET_RI EN €', 'MARKET_RI EN %',
       'MARKET_PVC NET', 'MARKET_LOT VIRTUEL',
       'MARKET_PRIX AU KG DU LOT VIRTUEL', 'FORMAT GV', 'FORMAT MV',
       'FORMAT MVK', 'PRODUIT DE UNE', 'PRODUIT EN DER', 'MISE EN AVANT',
       'VIDE_2', 'PICTO', 'SR', 'SR_SEGU_MARKET', 'CATALOG', 'CATALOG_MARKET',
       'AFFICHE', 'SELECTION FRANCAP', 'SELECTION SEGUREL',
       'INFO COMPLEMENTAIRES', 'PHOTO1', 'PHOTO2', 'PHOTO3', 'PHOTO4',
       'PHOTO5', 'PHOTO6', 'PHOTO7', 'PHOTO8', 'DESCRIPTIF_2', 'PRIX AU KG_2',
       'PRIX AU KG DU LOT VIRTUEL_2', 'DESCRIPTIF_3', 'PRIX AU KG_3',
       'PRIX A

25

In [26]:
for _, row in df_g.iterrows():
    row_to_duplicate = df.iloc[row.LISTE_INDEX[0]]
    row_to_duplicate.SR = False
    for i, index in enumerate(row.LISTE_INDEX):
        df.loc[index,'CATALOG'] = False
        row_to_duplicate[f"PHOTO{i+1}"]=df['PHOTO1'].iloc[index]
        row_to_duplicate[f"DESCRIPTIF_{i+1}"]=df.iloc[index]['DESCRIPTIF']
        row_to_duplicate[f"PRIX AU KG_{i+1}"]=df.iloc[index]['PRIX AU KG']
        row_to_duplicate[f"PRIX AU KG DU LOT VIRTUEL_{i+1}"]=df.iloc[index]['PRIX AU KG DU LOT VIRTUEL']

    row_df = pd.DataFrame([row_to_duplicate])    
    df_upper = df.iloc[:row.maxindex+1]
    df_lower = df.iloc[row.maxindex+1:]
    df = pd.concat([df_upper, row_df, df_lower]).reset_index(drop=True)

In [33]:
df[[
       'INTITULE', 'GENCOD', 'MARQUE', 'DESCRIPTIF', 'PRIX AU KG', 'PVC MAXI',
       'SR',  'CATALOG',
        'PHOTO1', 'PHOTO2', 'PHOTO3', 'DESCRIPTIF_2', 'PRIX AU KG_2',
       'PRIX AU KG DU LOT VIRTUEL_2', 'DESCRIPTIF_3', 'PRIX AU KG_3',
       'PRIX AU KG DU LOT VIRTUEL_3']][210:]

,INTITULE,GENCOD,MARQUE,DESCRIPTIF,PRIX AU KG,PVC MAXI,SR,CATALOG,PHOTO1,PHOTO2,PHOTO3,DESCRIPTIF_2,PRIX AU KG_2,PRIX AU KG DU LOT VIRTUEL_2,DESCRIPTIF_3,PRIX AU KG_3,PRIX AU KG DU LOT VIRTUEL_3
210,"Blé, riz rouge, quinoa",3175681157774,CEREAL BIO,Le doypack de 220 g,"Soit le kg : 10,41 €",2.29,False,True,3175681157774,3175681152137,,Le doypack de 220 g,"Soit le kg : 10,41 €",,None,None,None
211,"Blé, riz rouge, quinoa",3175681157774,CEREAL BIO,Le doypack de 220 g,"Soit le kg : 10,41 €",2.29,False,True,3175681157774,3175681152137,,Le doypack de 220 g,"Soit le kg : 10,41 €",,None,None,None
212,Biscuits diététiques,3251490332080,GERBLE,Figue & son \nLe paquet de 210 g\n,"Soit le kg : 7,38 €",1.55,True,True,3251490332080,,,None,None,None,None,None,None
213,Biscuits diététiques,3175681851849,GERBLE,Pomme noisette \nLe paquet de 230 g\n,"Soit le kg : 7,13 €",1.64,True,False,3175681851849,,,None,None,None,None,None,None
214,Thon entier au naturel cuisson vapeur,3019080063505,PETIT NAVIRE,La boîte de 130 g,"Soit le kg : 17,23 €",2.24,True,False,3019080063505,,,None,None,None,None,None,None
215,Biscuits diététiques,3175681851849,GERBLE,Pomme noisette \nLe paquet de 230 g\n,"Soit le kg : 7,13 €",1.64,False,True,3175681851849,3019080063505,,La boîte de 130 g,"Soit le kg : 17,23 €",,None,None,None
216,Biscuits diététiques,3175681851849,GERBLE,Pomme noisette \nLe paquet de 230 g\n,"Soit le kg : 7,13 €",1.64,False,True,3175681851849,3019080063505,,La boîte de 130 g,"Soit le kg : 17,23 €",,None,None,None
217,Secrets d'arôme Plein Sud,3011360002884,KNORR,La boîte de 60 g,"Soit le kg : 24,83 €",1.49,True,True,3011360002884,,,None,None,None,None,None,None
218,Assaisonnement bouquet garni,3011360005557,KNORR,9 cubes\nLa boîte de 99 g,"Soit le kg : 14,65 €",1.45,True,True,3011360005557,,,None,None,None,None,None,None
219,Plat cuisiné \nMon trio gourmand,3261055954114,WILLIAM SAURIN,Blanquette de volaille\nLa barquette de 320 g,"Soit le kg : 12,34 €",3.95,True,True,3261055954114,,,None,None,None,None,None,None
